In [61]:
#########################
# Homework 1 - Group 8: Scaravetti Alessio, Sterpin Enrico, Radinovic Jovana
# Part 2 Exercise 3
# We exted our environment in 2D, we create a 5x5 room, we add our reflex agent,  
# then we add some dirt and an obstacle.
# When the agent runs on the obstacle it will bump back on its previous location
# and change direction.
#########################

from agents import *
from notebook import psource
import random

# Initialization of the classes for the dirt and the obstacle
class Dirty(Thing):
    pass

class Obstacle(Thing):
    pass

# Initialization of the class for the room, we add a bit of graphics
class Room2D(GraphicEnvironment):

    def percept(self, agent):
        # return a list of things that are in our agent's location
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # target location
        # Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 2
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 2
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        # changes the state of the environment based on what the agent does.
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "Suck":
            items = self.list_things_at(agent.location, tclass=Dirty)
            if len(items) != 0:
                if agent.suck(items[0]): # the vacuum clean the floor
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) # delete it from the room
        elif action == 'bump':
            items = self.list_things_at(agent.location, tclass=Obstacle)
            if len(items) != 0:
                if agent.handle_bump(items[0]): 
                    print('{} bumped into a {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

def program(percepts):
    #Returns an action based on the vacuum's percepts
    for p in percepts:
        if isinstance(p, Dirty):
            return 'Suck'
        if isinstance(p, Bump):
            return 'turnright'
        if isinstance(p, Obstacle):
            return 'bump'
    # we decided to move the agent randomly    
    return random.choice(['turnright', 'turnleft', 'moveforward', 'moveforward', 'moveforward'])

class ReflexVacuum(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our vacuum is facing
    prev_location= None

    def moveforward(self, success=True):
        # we save the location in case of an obstacle
        self.prev_location=list(self.location)

        # moveforward possible only if success 
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1

    def turn(self, d):
        self.direction += d


    def suck(self, thing):
        # returns True upon success or False otherwise
        if isinstance(thing, Dirty):
            return True
        return False
    
    def handle_bump(self, thing):
        if isinstance(thing, Obstacle):
            if self.prev_location is not None:
                # the agent gets back on the previous location
                self.location = list(self.prev_location)
                self.prev_location = None  # reset of the previous location
                self.turn(Direction.R)
            return True
        return False


    
room = Room2D(5,5, color={'ReflexVacuum': (200,0,0), 'Dirty': (139, 69, 19), 'Obstacle': (0, 255, 0)}) # park width is set to 5, and height to 20
Vacu = ReflexVacuum(program)
dirt = Dirty()
obstacle=Obstacle()
room.add_thing(Vacu, [2,2])
room.add_thing(dirt, [random.choice([0, 1, 2, 3, 4]), random.choice([0, 1, 2, 3, 4])])
room.add_thing(obstacle, [0,2])
moredirt = Dirty()
room.add_thing(moredirt, [random.choice([0, 1, 2, 3, 4]), random.choice([0, 1, 2, 3, 4])])
room.run(20)


,,,,
,,,,
,,,,
,,,,
,,,,
